### 1 - Notebook for the Quant Track

### Getting started:
- Create a virtual environment with `python3.11 -m venv venv` (tested on Python 3.11, but Python 3.12 should be fine)
- Activate it with `source venv/bin/activate`
- Before running the notebook, please be sure to read the README.md in this folder, as it explains how to get data from BigQuery. In any case, pre-fetched data are already present in the `data` folder.
- Run the notebook (be sure to select the venv as kernel)

In [1]:
#This will install all the requirements 
!pip install -r requirements.txt

In [2]:
#Imports
import pandas as pd
import glob
import os
import json
from datetime import datetime, timedelta

The objective of this notebook is to analyze data from Aave V3 in the Polygon network for the tokens WETH and USDC.

The data is composed of two parts:
- **sample** contains data obtained from BigQuery providing `liquidity_rate`,`stable_borrow_rate`,`variable_borrow_rate`,`liquidity_index`,`variable_borrow_index` for both tokens.
In particular, the addresses are:
    - `0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174` for `usdc`
    - `0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619` for `weth`
- **volatility_data_june_2024** contains the assets value for `usdc` and `weth` obtained from the pycoingecko API.

Since the **sample** data are measured every minute, the first step is to standardize the volatility data into a table with a timestamp for every minute, aligning it with the sample data.

The output would be found in `../data/WETH_USDC_Prices.csv`.

In [3]:
# Function to read JSON data from a file
def read_prices_from_file(filename):
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
        return data['prices']
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from file '{filename}': {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# Function to create a DataFrame from price data
def create_price_dataframe(prices):
    # Convert timestamps to datetime and create a DataFrame
    timestamps = [datetime.fromtimestamp(item[0] / 1000) for item in prices]
    prices_values = [item[1] for item in prices]
    df = pd.DataFrame({'timestamp': timestamps, 'price': prices_values})
    df.set_index('timestamp', inplace=True)
    return df

# Read WETH and USDC prices
weth_prices = read_prices_from_file('../data/volatility_data_june_2024/aave-polygon-weth.json')
usdc_prices = read_prices_from_file('../data/volatility_data_june_2024/aave-polygon-usdc.json')

# Create DataFrames
weth_df = create_price_dataframe(weth_prices)
usdc_df = create_price_dataframe(usdc_prices)

# Define the time range for the entire month of June 2024
start_time = datetime(2024, 6, 1, 0, 0)
end_time = datetime(2024, 6, 30, 23, 59)
all_times = pd.date_range(start=start_time, end=end_time, freq='T')

# Initialize indices and current price holders
weth_idx = 0
usdc_idx = 0
weth_len = len(weth_df)
usdc_len = len(usdc_df)

# Create a results list
results = []

# Initialize with the first values from the DataFrames
current_weth = weth_df.iloc[0]['price'] if weth_len > 0 else None
current_usdc = usdc_df.iloc[0]['price'] if usdc_len > 0 else None

# Loop through each minute in the time range
for current_time in all_times:
    # Check for current WETH price if within bounds
    if weth_idx < weth_len and weth_df.index[weth_idx] <= current_time:
        current_weth = weth_df.iloc[weth_idx]['price']
        weth_idx += 1

    # Check for current USDC price if within bounds
    if usdc_idx < usdc_len and usdc_df.index[usdc_idx] <= current_time:
        current_usdc = usdc_df.iloc[usdc_idx]['price']
        usdc_idx += 1

    # Add the current time and prices to results
    results.append([current_time, current_weth, current_usdc])

# Create a final DataFrame
df = pd.DataFrame(results, columns=['timestamp', 'WETH', 'USDC'])

# Format the timestamp for the output
df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Save to CSV
df.to_csv('../data/WETH_USDC_Prices.csv', index=False, header=['', 'WETH', 'USDC'])

df.head()

/var/folders/ch/f52gc87d5_7b95m4hg7qfbj80000gn/T/ipykernel_15149/3214608902.py:34: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  all_times = pd.date_range(start=start_time, end=end_time, freq='T')


,timestamp,WETH,USDC
0,2024-06-01 00:00:00,3769.771717,1.00069
1,2024-06-01 00:01:00,3769.771717,1.00069
2,2024-06-01 00:02:00,3769.771717,1.00069
3,2024-06-01 00:03:00,3769.771717,1.00069
4,2024-06-01 00:04:00,3769.771717,1.00069


Now we can join the current dataframe with the liquidity and borrow rates for both tokens from the `sample` folder.

In [4]:
folder_path="../data/sample/"

# --- Step 1: Process 'weth' data ---

# Create an empty list to store the 'weth' DataFrames
weth_dfs = []

# Loop through all 'weth' CSV files in the folder
for file in glob.glob(folder_path + '*-0x7ceb23fd6bc0add59e62ac25578270cff1b9f619-*.minute.csv'):  # Filter for 'weth' files
    # Read the CSV file into a pandas DataFrame
    temp_df = pd.read_csv(file)

    # Rename columns by prepending 'weth'
    columns_to_rename = {
        'liquidity_rate': 'weth_liquidity_rate',
        'stable_borrow_rate': 'weth_stable_borrow_rate',
        'variable_borrow_rate': 'weth_variable_borrow_rate',
        'liquidity_index': 'weth_liquidity_index',
        'variable_borrow_index': 'weth_variable_borrow_index'
    }
    temp_df = temp_df.rename(columns=columns_to_rename)

    # Add the DataFrame to the list
    weth_dfs.append(temp_df)

# Concatenate all 'weth' DataFrames
weth_combined_df = pd.concat(weth_dfs)

# Merge the 'weth' DataFrame
weth_combined_df = pd.merge(df, weth_combined_df, left_on='timestamp', right_on='block_timestamp').drop("block_timestamp", axis=1)

weth_combined_df.head(5)

,timestamp,WETH,USDC,weth_liquidity_rate,weth_stable_borrow_rate,weth_variable_borrow_rate,weth_liquidity_index,weth_variable_borrow_index
0,2024-06-01 00:00:00,3769.771717,1.00069,0.004649,0.076128,0.020831,1.010326,1.043067
1,2024-06-01 00:01:00,3769.771717,1.00069,0.004649,0.076128,0.020831,1.010326,1.043067
2,2024-06-01 00:02:00,3769.771717,1.00069,0.004649,0.076128,0.020831,1.010326,1.043067
3,2024-06-01 00:03:00,3769.771717,1.00069,0.004649,0.076128,0.020831,1.010326,1.043067
4,2024-06-01 00:04:00,3769.771717,1.00069,0.004649,0.076128,0.020831,1.010326,1.043067


In [5]:
# --- Step 2: Process 'usdc' data ---

# Create an empty list to store the 'usdc' DataFrames
usdc_dfs = []

# Loop through all 'usdc' CSV files in the folder
for file in glob.glob(folder_path + '*-0x2791bca1f2de4661ed88a30c99a7a9449aa84174-*.minute.csv'):  # Filter for 'usdc' files
    # Read the CSV file into a pandas DataFrame
    temp_df = pd.read_csv(file)

    # Rename columns by prepending 'usdc'
    columns_to_rename = {
        'liquidity_rate': 'usdc_liquidity_rate',
        'stable_borrow_rate': 'usdc_stable_borrow_rate',
        'variable_borrow_rate': 'usdc_variable_borrow_rate',
        'liquidity_index': 'usdc_liquidity_index',
        'variable_borrow_index': 'usdc_variable_borrow_index'
    }
    temp_df = temp_df.rename(columns=columns_to_rename)
    
    # Add the DataFrame to the list
    usdc_dfs.append(temp_df)

# Concatenate all 'weth' DataFrames
usdc_combined_df = pd.concat(usdc_dfs)

# Merge the 'weth' DataFrame
all_combined_df = pd.merge(weth_combined_df, usdc_combined_df, left_on='timestamp', right_on='block_timestamp').drop("block_timestamp", axis=1)

all_combined_df.head(5)

all_combined_df.to_csv('../data/all_combined.csv', index=False)


Index(['timestamp', 'WETH', 'USDC', 'weth_liquidity_rate',
       'weth_stable_borrow_rate', 'weth_variable_borrow_rate',
       'weth_liquidity_index', 'weth_variable_borrow_index',
       'usdc_liquidity_rate', 'usdc_stable_borrow_rate',
       'usdc_variable_borrow_rate', 'usdc_liquidity_index',
       'usdc_variable_borrow_index'],
      dtype='object')

Now we are gonna use the 

The EMA Bearish Crossover is a technical analysis signal that indicates a potential downward trend or selling opportunity. It occurs when a short-term Exponential Moving Average (EMA) crosses below a longer-term EMA. This crossover suggests that the momentum is shifting from bullish (upward) to bearish (downward), which traders often interpret as a sell signal.

In [6]:
#Calculate the short term (10 days) EMA Bearish Crossover 

In [7]:
#Calculate the long term (100 days) EMA Bearish Crossover

If the 10-day EMA has crossed below the 100-day EMA, you could interpret it as a potential sell signal.
You then check the ADX:
-If ADX is above 25, you know the downtrend is strong, confirming the bearish move, and it may be a good opportunity to sell or reduce exposure.
-If ADX is below 20, the crossover might not be reliable, and you might decide to wait for more confirmation before acting.

In [8]:
#Calculate the short term (10 days) ADX 

In [9]:
#Calculate the short term (10 days) ADX 